<a href="https://colab.research.google.com/github/omid-sakaki-ghazvini/Projects/blob/main/persian_car_plates_digits_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 1. Install Dependencies and Setup

<div style="direction:rtl">
<font color='green' size="5px">
 کتابخانه های مورد نیاز را نصب میکنیم
    </font>
</div>

In [ ]:
pip install -q ultralytics
pip install -q wandb

In [ ]:
import wandb
wandb.init(mode="disabled")

import os                        # To work with operation system commands
import cv2                       # To process images
import random                    # to generate random choices
import warnings                  # To avoid python warnings
import numpy as np               # To work with arrays
import pandas as pd              # To work with DataFrames
import seaborn as sns            # To visualizations
from tqdm import tqdm            # To represent progress bars
from ultralytics import YOLO     # To Create Yolo model
from termcolor import colored    # To colorfull outputs
import matplotlib.pyplot as plt  # To visualizations

# ignore python warnings
warnings.filterwarnings('ignore')

# add style to plots
plt.style.use('fivethirtyeight')
sns.set_style('darkgrid')

# 2. Load Data

## https://www.kaggle.com/datasets/omidsakaki1370/persian-plates-digits

In [ ]:
import kagglehub

# Download latest version
path = kagglehub.dataset_download("omidsakaki1370/persian-plates-digits")

print("Path to dataset files:", path)

<div style="direction:rtl">
<font color='green' size="5px">
توسط خط فرمان زیر، دیتا را فراخوانی میکنیم
    </font>
</div>

In [ ]:
# Path of car-detection dataset
digits_base = '/kaggle/input/persian-plates-digits/Persian_Plates_Digits'

# Path of yaml file
digits_yaml_path = os.path.join(digits_base, 'data.yaml')

# Path of Train directory
digits_train_dir = os.path.join(digits_base, 'train')
digits_train_images = os.path.join(digits_train_dir, 'images')
digits_train_labels = os.path.join(digits_train_dir, 'labels')

# Path of Validation directory
digits_valid_dir = os.path.join(digits_base, 'valid')
digits_valid_images = os.path.join(digits_valid_dir, 'images')
digits_valid_labels = os.path.join(digits_valid_images, 'labels')

# Path of Test directory
digits_test_dir = os.path.join(digits_base, 'test')
digits_test_images = os.path.join(digits_test_dir, 'images')
digits_test_labels = os.path.join(digits_test_dir, 'labels')

In [ ]:
print(colored(f' Number of Train Images : {len(os.listdir(digits_train_images))} ', 'blue', attrs=['bold']))
print(colored(f' Number of Validation Images : {len(os.listdir(digits_valid_images))}', 'blue', attrs=['bold']))
print(colored(f' Number of Test Images : {len(os.listdir(digits_test_images))} ', 'blue', attrs=['bold']))
print('_'*40)
print(colored(f' Number of All Images : {len(os.listdir(digits_train_images)) + len(os.listdir(digits_valid_images)) + len(os.listdir(digits_test_images))} ', 'green',  attrs=['bold']))

In [ ]:
def Image_Size(train_images) :
    '''
    Read images and save image-sizes in a set
    '''
    img_sizes = set()

    # Loop over train images and read them
    for img_name in tqdm(os.listdir(train_images)) :
        img_path = os.path.join(train_images, img_name)
        img = cv2.imread(img_path)
        h, w, c = img.shape
        img_sizes.add((h, w))

    return img_sizes

In [ ]:
# Call the function in phase 1
img_sizes_digits = Image_Size(digits_train_images)

if len(img_sizes_digits) == 1 :
    print(colored(f'\n All the images have size of {img_sizes_digits} ', 'green', attrs=['bold']))
else :
    print(colored(f'\n There are {len(img_sizes_digits)} different image sizes ! ', 'red', attrs=['bold']))

In [ ]:
# Height and Width for all images
H_d = list(img_sizes_digits)[0][0]
W_d = list(img_sizes_digits)[0][1]

print(colored(f'Height : {H_d}\nWidth : {W_d}', 'black', attrs=['bold']))

In [ ]:
classes_dict =  {
            '0':'0',
            '1':'1',
            '2':'2',
            '3':'3',
            '4':'4',
            '5':'5',
            '6':'6',
            '7':'7',
            '8':'8',
            '9':'9',
            '10':'B',
            '11':'C',
            '12':'D',
            '13':'G',
            '14':'H',
            '15':'J',
            '16':'L',
            '17':'M',
            '18':'N',
            '19':'S',
            '20':'T',
            '21':'V',
            '22':'Y'
            }

## 3.Plot Random Images With BBOXs

In [ ]:
def draw_box(img, bboxes) :
    '''
    Read bboxes cordinates and calculate correct cordinates to draw rectangles.
    '''
    # bboxes contain 8 bbox for each image
    for i in range(len(bboxes)) :
        b1 = bboxes[i]
        b1 = b1.split(' ')

        # Decode the label index by classes_dict
        label = classes_dict[b1[0]]

        # Unnormalize centers
        x_center = float(b1[1]) * W_d
        y_center = float(b1[2]) * H_d
        w = float(b1[3]) * W_d
        h = float(b1[4]) * H_d

        # Determine 2 point of each box (top-left, bottom-right)
        x_min = round(x_center - (w / 2))
        x_max = round(x_center + (w / 2))
        y_min = round(y_center - (h / 2))
        y_max = round(y_center + (h / 2))

        # A small box above earlier boxes to write value in it
        x1 = round(x_center - w/4)
        x2 = round(x_center + w/4)
        y1 = round(3)
        y2 = round(10)

        # Draw boxes and write the values in them
        cv2.rectangle(img, (x_min, y_min), (x_max, y_max), (0, 0, 255), 1, cv2.LINE_AA)
        cv2.rectangle(img, (x1, y1), (x2, y2), (0, 0, 0), -1, cv2.LINE_AA)
        cv2.putText(img, label, (x1, y1+9), cv2.FONT_HERSHEY_COMPLEX, 0.35, (0, 255, 255), 1, cv2.LINE_AA)

In [ ]:
# Select K random images
random_images = random.choices(os.listdir(digits_train_images), k=12)
i = 1
plt.figure(figsize=(20, 7))
for img_name in random_images :
    plt.subplot(3, 4, i)
    # Image
    img_path = os.path.join(digits_train_images, img_name)
    img = cv2.imread(img_path)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    # # BBox
    txt_name = JPG_Name_To_TXT(img_name)
    lbl_path = os.path.join(digits_train_labels, txt_name)
    bboxes = open(lbl_path)
    bboxes = bboxes.readlines()

    draw_box(img, bboxes)

    plt.imshow(img)
    plt.axis('off')
    i += 1
plt.show()

In [ ]:
import wandb
wandb.init(mode="disabled")

# 4. Create a YoloV8n model

In [ ]:
# Create another yolo model and load the pretrained weights on it
yolo_model = YOLO('yolov8n.yaml').load('yolov8n.pt')

# Trian the model with train images and labels
result = yolo_model.train(data=digits_yaml_path, epochs=75, amp=False)

# 5.Train Result

In [ ]:


# List of some metrics
metrics = ['results.png','confusion_matrix.png', 'P_curve.png', 'PR_curve.png']

# Plot metrics images
plt.figure(figsize=(15, 12))
i = 1
for image_name in metrics:
    image_path = os.path.join(result_path, image_name)
    image = cv2.imread(image_path)

    plt.subplot(2, 2, i)

    plt.imshow(image)
    plt.title(image_name)
    i += 1

plt.show()



In [ ]:
# Read csv of result
df = pd.read_csv(os.path.join(result_path, 'results.csv'))
df.tail(5)

In [ ]:
# Strip columns
df.columns = df.columns.str.strip()
df.columns

In [ ]:
# Define X, y to plot
X = df.epoch
cols = df.columns[1:]

plt.figure(figsize=(15, 25))

for it in range(len(cols)) :
    plt.subplot(7, 2, it+1)
    ax = sns.lineplot(data=df, x=X, y=cols[it])
    ax.set_title(cols[it])
plt.suptitle('Training Metrics and Loss', fontsize=24)
plt.subplots_adjust(top=0.6)
plt.tight_layout()
plt.show()

## 6.Best weights for training, stored in weights folder as best.pt. load best weight in a model.

In [ ]:
# Path of working-Directory
working_dir = 'C:\\envs\\Working\\Iranian_Plates'

In [ ]:
# Location of stored result
result_path = result.save_dir

In [ ]:
# Load model with best weights
best_model_path = os.path.join(result_path, 'weights/best.pt')
best_model_digits = YOLO(best_model_path)

In [ ]:
# Evaluating the model
result_test = best_model_digits.val()

In [ ]:
# list of metrics
keys = ['metrics/precision(B)', 'metrics/recall(B)', 'metrics/mAP50(B)', 'metrics/mAP50-95(B)']

for key in keys :
    print(colored(f'{key} : {result_test.results_dict[key]}', 'green', attrs=['bold']))

## 7.Test the detection model

In [ ]:
# all of test images
images = os.listdir(digits_test_images)

num_samples = 12
random_images = random.choices(images, k=num_samples)

# Create a list of path of random test images.
images = []
for image in random_images :
    image_path = os.path.join(digits_test_images, image)
    images.append(image_path)

images

In [ ]:
# Predict labels and bboxes on random test images one by one
for image in images :
    result = best_model_digits.predict([image], save=True, conf=0.5, iou=0.7)

In [ ]:
# Location of saved predicted images
result_path = result[0].save_dir
print(result_path)

## 8.And plot the car images with predicted bboxes. Its a visual test

In [ ]:
# Show result images
i = 1
plt.figure(figsize=(12, 6), dpi=200)
for image in os.listdir(result_path) :
    image_path = os.path.join(result_path, image)
    plt.suptitle('Test Result', fontsize=25, fontweight=500)
    plt.subplot(2, 5, i)
    plt.imshow(plt.imread(image_path))
    plt.axis('off')
    plt.subplots_adjust(top=0.75)
    i+=1
    if i==11 : break
plt.show()